In [21]:
#imports
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import itertools
import os
import tweepy as tw
import pickle
from datetime import date

#Twitter API
import config
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\auggi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Read in Datafile
df = pd.read_csv('IMDBDataset.csv').to_numpy()
#temporary, cuts the dataset to 500 to save time running it when testing
df = df[:500]
size = int(df.size/2)

In [26]:
#Defines functions to be used in program

#Cleans HTML Input from string
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html) 
  return TextBlob(cleantext)

# This is how the Naive Bayes classifier expects the input
# removes stopwords from string
# returns a dictionary
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict

#Process Input
def findusefulwords(arr, sentiment):
    unique_words = {}
    useful_words = []

    for x in range (arr[0].size):
       unique_words[x] = set(cleanhtml(arr[0][x]).split(' '))

    for x in range (len(unique_words)):
        useful_words.append((create_word_features(unique_words[x]), sentiment))
    return useful_words

def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

#Search twitter API
def searchTwitter(search_string):
    today = date.today()
    d1 = today.strftime("%d/%m/%Y")
    tweets = tw.Cursor(api.search,
                       q=search_string,
                       lang="en",
                       since=d1).items(100)

    all_tweets = [remove_url(tweet.text) for tweet in tweets]
    return all_tweets

#Classify Tweets
def classifyTweets(tweets):
    return sentiment

In [4]:
#seperate data into pos and neg reviews, find the useful words for each and save to pos_words and neg_words
pos_reviews = []
neg_reviews = []

for x in range (size):
    if(df[x][1] == 'positive'):
        pos_reviews.append((df[x]))
    elif(df[x][1] == 'negative'):
        neg_reviews.append((df[x]))
pos_reviews = np.array(pos_reviews).T
pos_words = findusefulwords(pos_reviews,"positive")

neg_reviews = np.array(neg_reviews).T
neg_words = findusefulwords(neg_reviews,"negative")

#split data into training and testing datasets
train_set = neg_words[:200] + pos_words[:200]
test_set =  neg_words[200:] + pos_words[200:]

#Train the classifier and save it using pickle
#Classify and save to pickle
classifier = NaiveBayesClassifier.train(train_set)
# classifier.show_most_informative_features()
f = open('my_classifier.pickle', 'wb')
pickle.dump(classifier, f)
f.close()

In [3]:
#load classifier from pickle
f = open('my_classifier.pickle', 'rb')
classifier = pickle.load(f)
f.close()

In [4]:
#accuracy = nltk.classify.util.accuracy(classifier, test_set)
#print(accuracy * 100)

In [27]:
x = searchTwitter('Lebron')
print(x[:10])

['RT LakersNation Lakers will lean on their championship experience to get them through this rough stretch https', 'RT TotalProSports Welp', 'RT TotalProSports Welp', 'RT TotalProSports Welp', 'RT TotalProSports Welp', 'RT TotalProSports Welp', 'RT TotalProSports Welp', 'Welp', 'RT K1erry Linnies Has Never Been Busier Business is Booming For Cincinnati Pub that Banned NBA Games Because of LeBron James https', 'RT LakersNation Lakers will lean on their championship experience to get them through this rough stretch https']


In [ ]:
#TODO
#Clean Tweets, Classify them, Find overall sentiments, add the ablity to input search string, display overall sentiment data 

In [ ]:
#https://www.pythonforengineers.com/build-a-sentiment-analysis-app-with-movie-reviews/
#https://towardsdatascience.com/twitter-sentiment-analysis-classification-using-nltk-python-fa912578614c
#https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews